In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import csv

from datetime import datetime
from datetime import timedelta

#machine learning preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# machine learning sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from pandas.api.types import is_object_dtype, is_numeric_dtype

#machine learning pytorch


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
def rmse(y, yhat):
    return np.sqrt(mean_squared_error(y, yhat))

def dtype_as_category(df):

    for column in df.keys():
        if is_object_dtype(df[column]):
            df[column] = df[column].astype('category')

    return df

## 6. Correct Biases (graph neural network)
Note: For this section, only the raw data is relevant in order to minimize calulation times for the neural networks

### 6.1. Define node and node features (e.g. Turbines)
Lorem Ipsum

### 6.2 Define edge and edge features (e.g. Positional relationship)
Lorem Ipsum

In [4]:
g_dir : str         = "g_drive_data"
ip_dir : str        = "ip_forecast_data"
main_dir :str       = "main_data"

In [5]:
df = pd.read_csv(os.path.join(main_dir, "df_main_interpolated_fe.csv"))

In [8]:
pos_mapping : dict = {
    "front" : 0,
    "back" : 1,
}
df["position"] = df["position"].map(pos_mapping)
df = dtype_as_category(df)

In [10]:
#define the relevant columns for dataframes

relevant_all : list     = ["wt_id", "power_kw", "day_sin", "day_cos", "year_sin", "year_cos", "position", "month"]

relevant_3 : list       = [x for x in df.columns.tolist() if ("3" in x and "deviation" not in x)]
relevant_6 : list       = [x for x in df.columns.tolist() if ("6" in x and "deviation" not in x)]
relevant_12 : list      = [x for x in df.columns.tolist() if ("12" in x and "deviation" not in x)]

In [16]:
#creating the dfs for the relevant time frames
df_3        = df[relevant_all + relevant_3]
df_6        = df[relevant_all + relevant_6]
df_12       = df[relevant_all + relevant_12]

In [13]:
#creat dfs with raw forecast data

def get_raw_df(df : object, g_dir : str = g_dir, t : str = None) -> object:
    
    relevant_timestamps = pd.read_csv(os.path.join(g_dir, "forecasts_temp.csv"))
    data_points = pd.to_datetime(relevant_timestamps["init"]).dt.tz_localize(None)
    df_raw = df[pd.to_datetime(df[f"init_{t}"]).dt.tz_localize(None).isin(data_points)]

    print(f"df_{t}_raw:\t{df_raw.shape}")
    return df_raw

df_3_raw    = get_raw_df(df_3, g_dir = g_dir, t = 3)
df_6_raw    = get_raw_df(df_6, g_dir = g_dir, t = 6)
df_12_raw   = get_raw_df(df_12, g_dir = g_dir, t = 12)

for df_n in [df_3, df_3_raw, df_6, df_6_raw, df_12, df_12_raw]:
    cols = [x for x in df_n.columns.tolist() if "init" in x]
    df_n.drop(labels = cols, axis = 1,inplace = True)

del df_3, df_6, df_12 #freeing up memory

df_3_raw:	(28305, 14)
df_6_raw:	(28210, 14)
df_12_raw:	(28210, 14)


In [19]:
print(df_3_raw.columns.tolist())

['wt_id', 'power_kw', 'day_sin', 'day_cos', 'year_sin', 'year_cos', 'position', 'month', 'temp_3.00', 'wind_speed_3.00', 'wind_direction_3.00', 'density_3.00', 'power_3.00']
